# Welcome to Exkaldi

In this section, we will decode based on HMM-GMM model and WFST.

In [2]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Prepare a word-id table. We use the lexicons generated in early step directly. So load it. 

In [3]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

In lexicons, call "words" to get the word-id table if you want decode in words level. Or call "phones" to get the phone-ID table when decoding in phone level. But both them will return the Exkaldi ListTable object.

In [4]:
type(lexicons("words"))

exkaldi.core.achivements.ListTable

Prepare the acoustic feature for test. We compute the feature as same as training data.

In [5]:
scpFile = os.path.join(dataDir, "test", "wav.scp")
utt2spkFile = os.path.join(dataDir, "test", "utt2spk")
spk2uttFile = os.path.join(dataDir, "test", "spk2utt")

feat = exkaldi.compute_mfcc(scpFile, name="mfcc")
cmvn = exkaldi.compute_cmvn_stats(feat, spk2utt=spk2uttFile, name="cmvn")
feat = exkaldi.use_cmvn(feat, cmvn, utt2spk=utt2spkFile)

feat

Save it to file.

In [6]:
featFile = os.path.join(dataDir, "exp", "test_mfcc.ark")

feat.save(featFile, outScpFile=False)

'librispeech_dummy/exp/test_mfcc.ark'

Prepare the HMM-GMM model and WFST decoding graph. They have been generated in early steps.

In [7]:
HCLGFile = os.path.join(dataDir, "exp", "graph", "HCLG.fst")

hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")

Then decode. You can set some decoding parameters such as __beam__, __acwt__ and so on. Here we only use default configures.

In [8]:
lat = exkaldi.decode.wfst.gmm_decode(feat, hmmFile, HCLGFile, wordSymbolTable=lexicons("words"))

lat

gmm-latgen-faster --allow-partial=true --min-active=200 --max-active=7000 --max_mem=50000000 --beam=10 --lattice-beam=8 --acoustic-scale=1 --word-symbol-table=/tmp/tmpippec5t3_words.txt librispeech_dummy/exp/train_delta/final.mdl librispeech_dummy/exp/graph/HCLG.fst ark:- ark:- 
ERROR (gmm-latgen-faster[5.5.372~1-acff3]:LogLikelihoodZeroBased():decodable-am-diag-gmm.cc:50) Dim mismatch: data dim = 13 vs. model dim = 117

[ Stack-Trace: ]
kaldi::MessageLogger::LogMessage() const
kaldi::MessageLogger::LogAndThrow::operator=(kaldi::MessageLogger const&)
kaldi::DecodableAmDiagGmmUnmapped::LogLikelihoodZeroBased(int, int)
kaldi::DecodableAmDiagGmmScaled::LogLikelihood(int, int)
kaldi::LatticeFasterDecoderTpl<fst::Fst<fst::ArcTpl<fst::TropicalWeightTpl<float> > >, kaldi::decoder::StdToken>::ProcessEmitting(kaldi::DecodableInterface*)
kaldi::LatticeFasterDecoderTpl<fst::Fst<fst::ArcTpl<fst::TropicalWeightTpl<float> > >, kaldi::decoder::StdToken>::Decode(kaldi::DecodableInterface*)
bool kaldi:

KaldiProcessError: Failed to generate lattice.

___lat___ is an exkaldi __Lattice__ object. We will introduce it's property in detail in next step. Now, save it to file with kaldi format.

In [ ]:
outDir = os.path.join(dataDir, "exp", "decode_test")

exkaldi.utils.make_dependent_dirs(outDir, False)

In [ ]:
latFile = os.path.join(outDir "test.lat")

lat.save(latFile)